In [ ]:
from utilities import *
from pytorch_nn import Lipread2
from Model_training_blocks import FullTrainer
import torch
from torch import nn
from datasetloadingpy import dataloaders

In [ ]:
#Import all possible words as list
words_list_file = '/home/taylorpap/Bootcamp/wordlist.txt'
words_list = get_wordslist_from_txt_file(words_list_file)

In [ ]:
#Used for training From scratch (To generate weights before full scale)
temp_words_list = []
for index in range(1, len(words_list), 4):
    temp_words_list.append(words_list[index])
print(len(temp_words_list))

In [ ]:
#Create DataLoader for training model
data_path = '/media/taylorpap/2TBM2/Dataset_2/GreyCropped'
datasets = dataloaders(data_dir=data_path, label_fp=temp_words_list, batch_size=64, workers=16)

In [ ]:
#Create Model and specify number of classes
model = Lipread2(len(temp_words_list))
if torch.cuda.is_available():
    model.cuda()
#Specify the checkpoint to load (Used for second FullTrainer object below)
checkpoint_path = '/home/taylorpap/Bootcamp/Lipread2_full_scale/ckpt.pth.tar'
#Specify where checkpoint should be saved after each epoch
save_direc = '/home/taylorpap/Bootcamp/Lipread2_full_scale/'

In [ ]:
#Train model on smaller Dataset using larger batch size of 64, and number of epochs as 80 with a high learning rate for initial weights
train_model = FullTrainer(model, datasets, criterion=nn.CrossEntropyLoss(), epochs=80, save_dir=save_direc, lr=0.009)

Last Cell Output before cancel:
```
100%|██████████| 98/98 [00:07<00:00, 13.18it/s]
6249 in total	CR: 0.847655624899984
val Epoch:	33	Loss val: 0.5631	Acc val:0.8477, LR: 0.005890576474687263
Epoch len: 278.1077609062195 Estimated Remaining: 213.21595002810162 Min
Current Epoch: 34
```

In [ ]:
#Stopped Early to decrease learning rate, loading from checkpoint
train_model = FullTrainer(model, datasets, criterion=nn.CrossEntropyLoss().cuda(), epochs=80, save_dir=save_direc, state_path=checkpoint_path, lr=0.0009)

<font size='6'>
Scaled Model is made far smaller than initial attempts in order to speed up training time and efficiency of the model. This is done because of the huge scale of the dataset where time ends up being a large factor. In the real world, this model would also be expected to be fast enough for close to real time analysis, although another model would likely be needed for checking sentences of words as opposed to single word predictions.

In [ ]:
#Validated accuracy still acceptable on larger portion of dataset
#Using created weights to begin training full scale model
model = Lipread2(len(words_list))
#Move model to Cuda for faster training/performance
if torch.cuda.is_available():
    model.cuda()
data_path = '/media/taylorpap/2TBM2/Dataset_2/GreyCropped'
#Create new dataset object with full words list
datasets = dataloaders(data_dir=data_path, label_fp=words_list, batch_size=64, workers=16)
#Load above checkpoint for model weights. FullTrainer class uses only matching weights
checkpoint_path = '/home/taylorpap/Bootcamp/Lipread2_full_scale/ckpt.pth.tar'
save_direc = '/home/taylorpap/Bootcamp/Lipread2_actual_full_scale/'
train_model = FullTrainer(model, datasets, criterion=nn.CrossEntropyLoss().cuda(), epochs=80, save_dir=save_direc, state_path=checkpoint_path, model_weights_only=True, lr=0.009)

<font size='6'>
Despite the model being so lightweight, the total training time ended up being nearly 24 Hours total.

After training, Model Achieves about 80% Accuracy on full 500 word dataset. It is possible that extra layers could achieve a slightly higher score, but this implementation is fairly effecient and could be useful for ensembling multiple models together for the final result.
</font>